# Mapa Interactivo Crimen en Boston 2018

In [22]:
import pandas as pd
import geopandas as gpd
import math

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

Definimos una función embed_map () para mostrar mapas interactivos. Acepta dos argumentos: la variable que contiene el mapa y el nombre del archivo HTML donde se guardará el mapa.

Esta función asegura que los mapas sean visibles en todos los navegadores web.

In [23]:
# Function for displaying the map
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

### Tu primer mapa interactivo

Comenzamos creando un mapa relativamente simple con folium.Map ()

In [24]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# Display the map
embed_map(m_1, 'm_1.html')

# Los datos

Ahora, agregaremos algunos datos delictivos al mapa.

No nos centraremos en el paso de carga de datos. En cambio, puede imaginar que se encuentra en un punto en el que ya tiene los datos en un panda DataFrame `crime`. Las primeras cinco filas de los datos se muestran a continuación.

In [25]:
# Load the data
crimes = pd.read_csv("data/GEO/crimeA.csv", encoding='latin-1')

# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]
crimes = crimes[crimes.YEAR>=2018]

# Print the first five rows of the table
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,2018-09-03 21:25:00,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
19,I182070915,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,B2,181,NaN,2018-09-02 18:00:00,2018,9,Sunday,18,Part One,SHIRLEY ST,42.325695,-71.068168,"(42.32569490, -71.06816778)"
24,I182070908,522,Residential Burglary,BURGLARY - RESIDENTIAL - NO FORCE,B2,911,NaN,2018-09-03 18:38:00,2018,9,Monday,18,Part One,ANNUNCIATION RD,42.335062,-71.093168,"(42.33506218, -71.09316781)"


# Puntos de trazado

Para reducir la cantidad de datos que necesitamos incluir en el mapa, limitaremos (temporalmente) nuestra atención a los robos durante el día.

In [26]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(9,18))))]

## folium.Marker

Agregamos marcadores al mapa con `folium.Marker ()`. Cada marcador a continuación corresponde a un robo diferente.

In [27]:
# Create a map
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)

# Display the map
embed_map(m_2, 'm_2.html')

## folium.plugins.MarkerCluster

Si tenemos muchos marcadores para agregar, `folium.plugins.MarkerCluster ()` puede ayudar a ordenar el mapa. Cada marcador se agrega a un objeto `MarkerCluster`.

In [28]:
# Create the map
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))
m_3.add_child(mc)

# Display the map
embed_map(m_3, 'm_3.html')

# Mapas de burbujas

Un ** mapa de burbujas ** usa círculos en lugar de marcadores. Al variar el tamaño y el color de cada círculo, también podemos mostrar la relación entre la ubicación y otras dos variables.

Creamos un mapa de burbujas usando `folium.Circle ()` para agregar círculos iterativamente. En la celda de código a continuación, los robos que ocurrieron en las horas 9-12 se trazan en verde, mientras que los robos de las horas 13-17 se trazan en rojo.

In [29]:
# Create a base map
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map
for i in range(0,len(daytime_robberies)):
    Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=20,
        color=color_producer(daytime_robberies.iloc[i]['HOUR'])).add_to(m_4)

# Display the map
embed_map(m_4, 'm_4.html')

Tenga en cuenta que `folium.Circle ()` toma varios argumentos:
- `ubicación` es una lista que contiene el centro del círculo, en latitud y longitud.
- `radio` establece el radio del círculo.
  - Tenga en cuenta que en un mapa de burbujas tradicional, el radio de cada círculo puede variar. Podemos implementar esto definiendo una función similar a la función `color_producer ()` que se usa para variar el color de cada círculo.
- `color` establece el color de cada círculo.
  - La función `color_producer ()` se utiliza para visualizar el efecto de la hora en la ubicación del robo.

# Mapas de calor

Para crear un mapa de calor, utilizamos [`folium.plugins.HeatMap ()`] (https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap). Esto muestra la densidad del crimen en diferentes áreas de la ciudad, donde las áreas rojas tienen relativamente más incidentes criminales.

Como era de esperar para una gran ciudad, la mayor parte del crimen ocurre cerca del centro.

In [15]:
# Create a base map
m_5 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=crimes[['Lat', 'Long']], radius=10).add_to(m_5)

# Display the map
embed_map(m_5, 'm_5.html')

Como puede ver en la celda de código anterior, `folium.plugins.HeatMap ()` toma un par de argumentos:
- `data` es un DataFrame que contiene las ubicaciones que nos gustaría trazar.
- `radio` controla la suavidad del mapa de calor. Los valores más altos hacen que el mapa de calor parezca más suave (es decir, con menos huecos).

# Mapas coropléticos

Para comprender cómo varía el crimen según el distrito policial, crearemos un mapa coroplético.

Como primer paso, creamos un GeoDataFrame donde a cada distrito se le asigna una fila diferente, y la columna "geometría" contiene los límites geográficos.

In [17]:
# GeoDataFrame with geographical boundaries of Boston police districts
districts_full = gpd.read_file('data/GEO/Police_Districts.shp')
districts = districts_full[["DISTRICT", "geometry"]].set_index("DISTRICT")
districts.head()

,geometry
DISTRICT,
A15,"MULTIPOLYGON (((-71.07416 42.39051, -71.07415 ..."
A7,"MULTIPOLYGON (((-70.99644 42.39557, -70.99644 ..."
A1,"POLYGON ((-71.05200 42.36884, -71.05169 42.368..."
C6,"POLYGON ((-71.04406 42.35403, -71.04412 42.353..."
D4,"POLYGON ((-71.07416 42.35724, -71.07359 42.357..."


In [18]:
# Number of crimes in each police district
plot_dict = crimes.DISTRICT.value_counts()
plot_dict.head()

D4     2885
B2     2231
A1     2130
C11    1899
B3     1421
Name: DISTRICT, dtype: int64

Es muy importante que `plot_dict` tenga el mismo índice que` districts`; así es como el código sabe cómo hacer coincidir los límites geográficos con los colores apropiados.

Usando la clase `folium.Choropleth ()`, podemos crear un mapa coroplético.

In [20]:
# Create a base map
m_6 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
Choropleth(geo_data=districts.__geo_interface__, 
           data=plot_dict, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(m_6)

# Display the map
embed_map(m_6, 'm_6.html')

Tenga en cuenta que `folium.Choropleth ()` toma varios argumentos:
- `geo_data` es una GeoJSON FeatureCollection que contiene los límites de cada área geográfica.
  - En el código anterior, convertimos el GeoDataFrame de `distritos` en una [GeoJSON FeatureCollection] (https://en.wikipedia.org/wiki/GeoJSON) con el atributo` __geo_interface__`.
- `data` es una serie Pandas que contiene los valores que se utilizarán para codificar por color cada área geográfica.
- `key_on` siempre se establecerá en` feature.id`.
  - Esto se refiere al hecho de que el GeoDataFrame utilizado para `geo_data` y la Serie Pandas proporcionada en` data` tienen el mismo índice. Para comprender los detalles, tendríamos que observar más de cerca la estructura de una Colección de características GeoJSON (donde el valor correspondiente a la tecla "características" es una lista, en donde cada entrada es un diccionario que contiene una clave "id").
- `fill_color` establece la escala de color.
- `legend_name` etiqueta la leyenda en la esquina superior derecha del mapa.